### Augment the PythonPath so python can find necessary code.

In [ ]:
import os, sys, datetime
LF_CODE_PATH = os.path.expanduser('~/projects/LFAnalyze/code')
FT_CODE_PATH = os.path.expanduser('~/projects/fishTrax/code/analysis/')
FD_CODE_PATH = os.path.expanduser('~/projects/fish_despair_notebooks/src/')
sys.path.insert(0,LF_CODE_PATH)
sys.path.insert(0,FT_CODE_PATH)
sys.path.insert(0,FD_CODE_PATH)

### Import useful python packages

In [ ]:
%matplotlib inline
import matplotlib as mpl
import matplotlib.pyplot as plt
import pims
import tqdm
import numpy as np
import pandas as pd
import scipy
import skimage.io
import visualization_utils as vizutil
import seaborn as sns
from skimage.filters import gaussian_filter

### Load the data

passivity_2p_imaging_utils provides a list of all the datasets and provides a helper class to make loading the data easy.

The data_sets are split into three conditions... (the keynames are wierd for historical reasons)   
'enp': control group - no shocks at all.  
'c': experimental group - fish experience behavioral challenge (repeated shocks) while being imaged  
'e': reexposed group - first fish experience free-swimming behavioral challenge... then are imaged during behavioral challenge
  
all_data is a dictionary keyed by the condition.  
all_data[condition] contains a list of Passivity_2p_Fish objects.

In [ ]:
import passivity_2p_imaging_utils as p2putils
reload(p2putils)
tmp_dir = '/tmp/'
all_data = p2putils.get_all_datasets(tmp_dir=tmp_dir)

print 'controls, n =', len(all_data['enp'])
print 'experimental, n =', len(all_data['c'])
print 'reexposed, n =', len(all_data['e'])

Some ideal example fish for modeling
(see 2p_Fish_Figure3_Accumulation.ipynb)

In [ ]:
print 'experimental', all_data['c'][6].fishid
print 'reexposed', all_data['e'][2].fishid

### Passivity_2p_Fish Class

The `Passivity_2p_Fish` class offers various members and methods for loading data associated with each fish.  
  
Lets demo these methods using an example fish:  

In [ ]:
f = all_data['e'][2]

The time of all volumes, tail_movements, and shocks are stored in the following variable:

In [ ]:
print 'The first shock started at t=', f.shock_st[0], 's'#shock start times
print 'The first slice of the frame began being imaged at t=', f.frame_st[0,0], 's' #time at which each slice was imaged [#samples X #slices]
print 'The first tail movement started at=', f.tail_movement_start_times[0], 's' #tail movement times - movements separted into forward swims, turns and escapes.
print 'Num z-planes imaged:', f.num_zplanes
print 'Volume-Rate:', 1/np.diff(f.frame_st[:,0]).mean() #frame_st is #frames x #slices, we examine interval between imaging first slice

Demo code for examining movie of tail (slowed by factor of 10)

In [ ]:
movement_ndx = 10
clip = f.get_tail_movie_clip(f.tail_movement_start_times[movement_ndx]-.1, 
                             f.tail_movement_end_times[movement_ndx]+.1, 
                             playback_speed_factor=.1)
clip.ipython_display(width=150)

Demo code to plot tail movement rate

In [ ]:
rate_window_starts = np.arange(-300,31*60,60) + f.shock_st[0] #1 minutes windows around start of shock
rate_window_centers = rate_window_starts + 30
rate_windows_adj = rate_window_starts - f.td_time[0] #corrected for the fact that get_movement_rate wants time relative to start of tail imaging.
plt.plot(rate_window_centers - f.shock_st[0], [f.get_movement_rate([x,x+60], bExcShockResp=False) for x in rate_windows_adj])
plt.axvline(0,c='r')
plt.ylabel('Movement Rate (Hz)')
plt.xlabel('Time Relative to First Shock (s)')

The `get_roi_table` method returns a dataframe of all the rois for teh fish.  Each row of this table represents an ROI and specified the place the ROI is in, the pixels that are included in the ROI, the centroid of the ROI, and which brain regions the ROI is in.  

Note, this data is older and was processed by simply segmenting the anatomical images.  Thus the data is does not look as clean as data that is cleaned up and processed using CNMF, for example.

In [ ]:
df = f.get_roi_table() #this can be slow to run the first time as data is loaded from files
df.head()

'get_signals_raw' returns a matrix containing the raw fluorescent signal associated with each ROI.  Each row of this matrix is associated with the corresponding row of the ROI table.  

Note, I only use the second half of the signal matrix, because the agarose had not fully hardened during the first of imaging which cause the fish to drift in z slightly.

In [ ]:
M = f.get_signals_raw(z=None)
#M = hbutils.df_over_f(M)
print 'Num ROIs:', df.shape[0]
print 'Shape of signal matrix', M.shape

There are also various methods for grabbing the raw imaging data:  
get_tif_as_vol  
get_tif_rasl

We can use this to visualize a few ROIs in particular plane/slice and brain region:

(**This won't work on AWS unless you upload tif stacks**)

In [ ]:
z=8

#Create a background image by averaging 200 frames and adjusting the gamma.
back_img = np.power(f.get_tif_rasl_as_vol(z,range(1,200)).mean(axis=2),.4)

#Select rois in raphe in this slices, and get their coordinates.
coords = df[(df.in_r_raphe) & (df.z==z)].coords

#Overlay the ROIs on the background image and display:
img = vizutil.overlay_coords(back_img, coords, [0,0,1], alpha=.5)
plt.figure(figsize=[20,20])
plt.imshow(img,interpolation='nearest')